# Price Data Extraction for Post-Index Rebalancing Arbitrage Strategy

Data source: Refinitiv Datastream via WRDS

This notebook executes codes to extract relevant data, based on the historical records of FTSE100 and FTSE250 rebalancing, for the past 10 years (2013Q1 - 2023Q3).
In this exercise, the following stocks are excluded:
- Stocks that are suspended from trading within the analysis period (+/- 20 days from rebalancing date)
- Stocks which rebalancing dates fall within the announcement date and the ex date of a corporate action
- Stocks that we are unable to obtain a reliable historical data on
- All Q3 2023 rebalancing; at time of study we are unable to obtain 20 days after the rebalancing date

In [1]:
# Import WRDS library
import wrds
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os

## Data Extraction

In [2]:
def read_sql_script(fname):
    fd = open(fname, 'r')
    sqlFile = fd.read()
    fd.close()

    return sqlFile


# Get current path
current_dir = os.getcwd()


# Define sql file names
# these will be used as a global variable
query_historical_prices = read_sql_script('/Users/abigail/Desktop/SMU/QF603/get_historical_prices.sql')
query_shares_outstanding = read_sql_script('/Users/abigail/Desktop/SMU/QF603/get_shares_outstanding.sql')

# Establish live connection; requires user login (passwords will be masked)
db = wrds.Connection() # this will be used as a global variable


def get_historical_prices(isin, start_date, end_date):
    
    print(f'Extracting historical prices for {isin}...')

    df =\
    (
        db
        .raw_sql(
            query_historical_prices.format(isin, start_date, end_date), 
            date_cols = ['trade_date']
            )
    )

    if df.empty:
        print('Dataframe is empty. No results was returned!')
    
    print('--------------------------------------------------')

    return df


Enter your WRDS username [abigail]:abigailcjh
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
ftse_rebal = pd.read_csv('/Users/abigail/Desktop/SMU/QF603/ftse_10y_rebal_records.csv')
# ftse_rebal.head()

# hist_rebal_price = pd.read_csv('')

In [4]:
look_back = 40
look_forward = 40

ftse_rebal["Post Date"] =\
    pd.to_datetime(ftse_rebal["Post Date"], 
                   format = '%d/%m/%Y')

ftse_rebal["start_date"] =\
    (
        ftse_rebal["Post Date"] - timedelta(days = look_back)
    ).dt.strftime('%d/%m/%Y')

ftse_rebal["end_date"] =\
    (
        ftse_rebal["Post Date"] + timedelta(days = look_back)
    ).dt.strftime('%d/%m/%Y')

In [5]:
target_isins = ftse_rebal["ISIN"]
start_dates = ftse_rebal["start_date"]
end_dates = ftse_rebal["end_date"]   


historical_prices =\
    (
        pd.
        concat(
            map(
                get_historical_prices,
                target_isins,
                start_dates,
                end_dates
            )
        )
    )

Extracting historical prices for GI000A0F6407...
--------------------------------------------------
Extracting historical prices for GB00BF8Q6K64...
--------------------------------------------------
Extracting historical prices for GB00BM8NFJ84...
--------------------------------------------------
Extracting historical prices for GB00BMCYKB41...
--------------------------------------------------
Extracting historical prices for GB00B23K0M20...
--------------------------------------------------
Extracting historical prices for GB00BG5KQW09...
--------------------------------------------------
Extracting historical prices for GB00B14SKR37...
--------------------------------------------------
Extracting historical prices for GB0009633180...
--------------------------------------------------
Extracting historical prices for GB0001826634...
--------------------------------------------------
Extracting historical prices for GG00BMD8MJ76...
--------------------------------------------------


--------------------------------------------------
Extracting historical prices for GG00B1ZBD492...
--------------------------------------------------
Extracting historical prices for GB0031544546...
--------------------------------------------------
Extracting historical prices for JE00B6T5S470...
--------------------------------------------------
Extracting historical prices for GB0009039941...
--------------------------------------------------
Extracting historical prices for GB00B018CS46...
--------------------------------------------------
Extracting historical prices for GB0030474687...
--------------------------------------------------
Extracting historical prices for GB00BMV92D64...
--------------------------------------------------
Extracting historical prices for GB0001500809...
--------------------------------------------------
Extracting historical prices for GB00BYV8MN78...
--------------------------------------------------
Extracting historical prices for GB00BJTNFH41...


--------------------------------------------------
Extracting historical prices for GB0002418548...
--------------------------------------------------
Extracting historical prices for GB00BYYW3C20...
--------------------------------------------------
Extracting historical prices for IM00B5VQMV65...
--------------------------------------------------
Extracting historical prices for GB00BYYTFB60...
--------------------------------------------------
Extracting historical prices for GB00BKP36R26...
--------------------------------------------------
Extracting historical prices for GG00BJL5FH87...
--------------------------------------------------
Extracting historical prices for GB0003450359...
--------------------------------------------------
Extracting historical prices for GB0033195214...
--------------------------------------------------
Extracting historical prices for GB0007388407...
--------------------------------------------------
Extracting historical prices for GB00B1JQDM80...


--------------------------------------------------
Extracting historical prices for GB00B03HDJ73...
--------------------------------------------------
Extracting historical prices for GB00BFZNLB60...
--------------------------------------------------
Extracting historical prices for GB0002945029...
--------------------------------------------------
Extracting historical prices for IM00B5VQMV65...
--------------------------------------------------
Extracting historical prices for GB00B012TP20...
--------------------------------------------------
Extracting historical prices for GB00BKX5CN86...
--------------------------------------------------
Extracting historical prices for GB0004915632...
--------------------------------------------------
Extracting historical prices for GB00BJ62K685...
--------------------------------------------------
Extracting historical prices for GB00BGXQNP29...
--------------------------------------------------
Extracting historical prices for GB00B60BD277...


--------------------------------------------------
Extracting historical prices for GB00BLJNXL82...
--------------------------------------------------
Extracting historical prices for GB0007365546...
--------------------------------------------------
Extracting historical prices for GB00B7FC0762...
--------------------------------------------------
Extracting historical prices for GB00B41H7391...
--------------------------------------------------
Extracting historical prices for BMG702782084...
--------------------------------------------------
Extracting historical prices for GB00B1Z4ST84...
--------------------------------------------------
Extracting historical prices for GB00BDVZYZ77...
--------------------------------------------------
Extracting historical prices for GG00BV54HY67...
--------------------------------------------------
Extracting historical prices for GB00BLRLH124...
--------------------------------------------------
Extracting historical prices for GB00BJTNFH41...


--------------------------------------------------
Extracting historical prices for GB00B1QH8P22...
--------------------------------------------------
Extracting historical prices for GB00BVGBWW93...
--------------------------------------------------
Extracting historical prices for IE0002424939...
--------------------------------------------------
Extracting historical prices for GB00BCKFY513...
--------------------------------------------------
Extracting historical prices for GB00B01FLG62...
--------------------------------------------------
Extracting historical prices for GB00BYRJH519...
--------------------------------------------------
Extracting historical prices for GB0004478896...
--------------------------------------------------
Extracting historical prices for GB00BYXJC278...
--------------------------------------------------
Extracting historical prices for GB00B0HZPV38...
--------------------------------------------------
Extracting historical prices for GB0005758098...


--------------------------------------------------
Extracting historical prices for GB00BNGWY422...
--------------------------------------------------
Extracting historical prices for GB00BMQX2Q65...
--------------------------------------------------
Extracting historical prices for GB0001570810...
--------------------------------------------------
Extracting historical prices for GB00BMHTHT14...
--------------------------------------------------
Extracting historical prices for GB00B1YW4409...
--------------------------------------------------
Extracting historical prices for GI000A0F6407...
--------------------------------------------------
Extracting historical prices for GB00BJTNFH41...
--------------------------------------------------
Extracting historical prices for GB00BKRV3L73...
--------------------------------------------------
Extracting historical prices for GB0004228648...
--------------------------------------------------
Extracting historical prices for GB0006834344...


--------------------------------------------------
Extracting historical prices for GB0004866223...
--------------------------------------------------
Extracting historical prices for GB00B0SWJX34...
--------------------------------------------------
Extracting historical prices for GB00B7FC0762...
--------------------------------------------------
Extracting historical prices for GB00B0D5V538...
--------------------------------------------------
Extracting historical prices for GB00B03HDJ73...
--------------------------------------------------
Extracting historical prices for GB00B1VYCH82 ...
Dataframe is empty. No results was returned!
--------------------------------------------------


/var/folders/mv/kbtg0d9s34s4w1xjw4w31b0h0000gn/T/ipykernel_80017/4165491479.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.


In [8]:
historical_prices.head()

,trade_date,security_code,security_name,primary_exchange,refinitiv_code,isin_code,currency,open,high,low,close,volume
0,2023-08-09,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.066,1.136,1.063000,1.117,358252.0
1,2023-08-10,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.140,1.140,1.091000,1.120,284950.0
2,2023-08-11,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.100,1.331,1.098000,1.150,1003896.0
3,2023-08-14,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.147,1.160,1.088108,1.096,1088784.0
4,2023-08-15,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.100,1.126,1.001000,1.114,1127118.0


## Data Cleaning

In [61]:

index_close_data = pd.read_csv('/Users/abigail/Desktop/SMU/QF603/FTSE_100_Index_10y.csv', header = 0)

index_close_data["Date"] =\
    (
        index_close_data["Date"]
        .apply(lambda x: datetime.strptime(x,'%d/%m/%y'))
    )


In [62]:
index_close_data.head()

,Date,Close,Net,%Chg,Open,Low,High,Volume,Turnover - GBP,Flow
0,2012-02-17,5905.07,19.69,0.33%,"5,885.38","5,885.38","5,923.62","1,101,910,955","405,068.00","-404,602.00"
1,2012-02-20,5945.25,40.18,0.68%,"5,905.07","5,905.07","5,956.33","723,669,610","270,963.00","-133,639.00"
2,2012-02-21,5928.20,-17.05,-0.29%,"5,945.25","5,916.58","5,948.84","846,269,111","326,004.00","-459,643.00"
3,2012-02-22,5916.55,-11.65,-0.20%,"5,928.20","5,894.60","5,937.96","872,088,943","327,016.00","-786,659.00"
4,2012-02-23,5937.89,21.34,0.36%,"5,916.55","5,900.50","5,952.47","1,071,356,849","340,406.00","-446,253.00"


In [10]:
full_ftse_data = historical_prices.copy()

## HEREEEE

In [64]:
# Getting list of ftse close by trade date 

close_price = []
trade_date = list(full_ftse_data.trade_date)
count = -1

for date in trade_date:
    count = count  + 1
    if index_close_data.index[(index_close_data["Date"] == date)].empty:
        look_back = 1
        while look_back < 10: 
            if index_close_data.index[(index_close_data["Date"] == date)].empty:
                date = (date - timedelta(days = look_back))
            else:
                idx = index_close_data.index[(index_close_data["Date"] == date)][0]
                close_price.append(index_close_data["Close"][idx])
                break
            looks_back = look_back + 1
    else:
        idx = index_close_data.index[(index_close_data["Date"] == date)][0]
        close_price.append(index_close_data["Close"][idx])


In [67]:
# Adding list to dataframe
full_ftse_data["index_close"] = close_price

In [69]:
# Here, we only filter for stocks that are listed on LSEG
# There are stocks that somehow the datastream returns the primary stock listed on other exchanges
# Those stocks should not be part of the analysis

lse_historical_prices = full_ftse_data.loc[full_ftse_data.primary_exchange == 'LON', :].copy()
lse_historical_prices.close.isna().sum()

0

In [70]:
rebal_round = {
    1 : 'Q4',
    2 : 'Q1',
    3 : 'Q1',
    4 : 'Q1',
    5 : 'Q2',
    6 : 'Q2',
    7 : 'Q2',
    8 : 'Q3',
    9 : 'Q3',
    10 : 'Q3',
    11 : 'Q4',
    12 : 'Q4',
}

In [71]:
lse_historical_prices['year'] = lse_historical_prices['trade_date'].dt.year
lse_historical_prices['month'] = lse_historical_prices['trade_date'].dt.month
lse_historical_prices['rebal'] =\
(
    (lse_historical_prices['year'] 
     - 1*(lse_historical_prices['month'] == 1)).astype(str)
    + lse_historical_prices['month'].map(rebal_round)
)

lse_historical_prices.head()

,trade_date,security_code,security_name,primary_exchange,refinitiv_code,isin_code,currency,open,high,low,close,volume,index_close,year,month,rebal
0,2023-08-09,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.066,1.136,1.063000,1.117,358252.0,7587.30,2023,8,2023Q3
1,2023-08-10,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.140,1.140,1.091000,1.120,284950.0,7618.60,2023,8,2023Q3
2,2023-08-11,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.100,1.331,1.098000,1.150,1003896.0,7524.16,2023,8,2023Q3
3,2023-08-14,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.147,1.160,1.088108,1.096,1088784.0,7507.15,2023,8,2023Q3
4,2023-08-15,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.100,1.126,1.001000,1.114,1127118.0,7389.64,2023,8,2023Q3


In [72]:
# Remove Q3 2023 Rebal due to incomplete data
lse_historical_prices =\
    lse_historical_prices[lse_historical_prices['rebal'] != '2023Q3']

lse_historical_prices.head()

,trade_date,security_code,security_name,primary_exchange,refinitiv_code,isin_code,currency,open,high,low,close,volume,index_close,year,month,rebal
0,2023-05-10,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,6.310000,6.390098,4.873999,4.873999,3884045.0,7741.33,2023,5,2023Q2
1,2023-05-11,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,4.917998,5.159998,4.449998,5.000000,6081492.0,7730.58,2023,5,2023Q2
2,2023-05-12,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,5.050000,5.482000,4.964507,5.050000,3685914.0,7754.62,2023,5,2023Q2
3,2023-05-15,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,4.649998,4.800000,3.805950,4.005000,5816469.0,7777.70,2023,5,2023Q2
4,2023-05-16,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,3.960000,4.273999,3.932000,3.988999,1900093.0,7751.08,2023,5,2023Q2


In [73]:
lse_historical_prices =\
    lse_historical_prices\
    .sort_values(by = ['security_name', 'trade_date'])\
    .reset_index(drop = True)

## Get FTSE100/Stock to get Beta
Generate FTSE 100 Index Close based on Trade dates of Historical Price DF - get covariance for stock & index and variance for index

## ERRORS: 
- Beta looks weird

In [ ]:
def get_return(price_list):
    length = len(price_list) -1
    t1 = price_list[0:length]
    t2 = price_list[1:]

    result = [(j / i -1) for i, j in zip(t1, t2)]
    
    return result

In [113]:
target_isins= ftse_rebal["ISIN"]
rebal_dates = ftse_rebal["Post Date"].dt.strftime('%d/%m/%Y')

stock_price = list(full_ftse_data["close"])
index_close = list(full_ftse_data["index_close"])

target_rebal_prices = []


for isin, rebal_date in zip(target_isins, rebal_dates):
    # Remove stocks that are suspended from trading during the analysis period
    if (isin, rebal_date) in [('GB00BJP5HK17', '19/12/2022'), 
                              ('GB00B1VNST91', '18/06/2018'),
                              ('GB0007892358', '19/06/2017')]:
        continue
    if lse_historical_prices[(lse_historical_prices.isin_code == isin) 
                             & (lse_historical_prices.trade_date == rebal_date)].empty:
        print(f'ISIN {isin} for {rebal_date} is excluded from studies!')
    
    else:
        sub_df = lse_historical_prices[lse_historical_prices.isin_code == isin]
        rebal_idx =\
        (
            sub_df
            .index[sub_df.trade_date == rebal_date]
            [0]
        )
        
        for delta in [-20, -5, -3, -1, 3, 5, 10, 20]:
            # Ensure that the prices for the days required exist
            
            assert sub_df['rebal'].loc[rebal_idx] == sub_df['rebal'].loc[rebal_idx + delta],\
            f'ISIN {isin} faced insufficient data pre-rebal on {rebal_date} for delta {delta} days'
        
        pre_20_pd = sub_df.close.loc[rebal_idx - 20]
        pre_5_pd = sub_df.close.loc[rebal_idx - 5]
        pre_3_pd = sub_df.close.loc[rebal_idx - 3]
        pre_1_pd = sub_df.close.loc[rebal_idx - 1]
        post_3_pd = sub_df.close.loc[rebal_idx + 3]
        post_5_pd = sub_df.close.loc[rebal_idx + 5]
        post_10_pd = sub_df.close.loc[rebal_idx + 10]
        post_20_pd = sub_df.close.loc[rebal_idx + 20]
        
        start_idx = rebal_idx - 20
        
        lst_stock_price = get_return(stock_price[start_idx:rebal_idx])
        lst_index_close = get_return(index_close[start_idx:rebal_idx])
        lst_joint = []
        lst_joint.append(lst_stock_price)
        lst_joint.append(lst_index_close)
        
        cov = np.cov(lst_joint)[0][1]
        var = np.var(lst_index_close)
        
        beta = cov/var
        corr = np.corrcoef(lst_joint)[0][1]
        
        target_rebal_prices.append({
            'Name' : sub_df.security_name.values[0],
            'ISIN' : isin,
            'post_date' : rebal_date,
            'pre_twenty_pd' : pre_20_pd,
            'pre_five_pd' : pre_5_pd,
            'pre_three_pd' : pre_3_pd,
            'pre_one_pd' : pre_1_pd,
            'post_three_pd' : post_3_pd,
            'post_five_pd' : post_5_pd,
            'post_ten_pd' : post_10_pd,
            'post_twenty_pd' : post_20_pd,
            'Cov': cov,
            'Var': var,
            'Beta': beta,
            'Corr': corr,
            
        })
    

ISIN GI000A0F6407 for 18/09/2023 is excluded from studies!
ISIN GB00BF8Q6K64 for 18/09/2023 is excluded from studies!
ISIN GB00BM8NFJ84 for 18/09/2023 is excluded from studies!
ISIN GB00BMCYKB41 for 18/09/2023 is excluded from studies!
ISIN GB00B23K0M20 for 18/09/2023 is excluded from studies!
ISIN GB00BG5KQW09 for 18/09/2023 is excluded from studies!
ISIN GB00B14SKR37 for 18/09/2023 is excluded from studies!
ISIN GB0009633180 for 18/09/2023 is excluded from studies!
ISIN GB0001826634 for 18/09/2023 is excluded from studies!
ISIN GG00BMD8MJ76 for 18/09/2023 is excluded from studies!
ISIN GB00B0LCW083 for 18/09/2023 is excluded from studies!
ISIN BMG4593F1389 for 18/09/2023 is excluded from studies!
ISIN GB00BZ4BQC70 for 18/09/2023 is excluded from studies!
ISIN GB0031274896 for 18/09/2023 is excluded from studies!
ISIN GB00BY7QYJ50 for 18/09/2023 is excluded from studies!
ISIN GB00BMT9K014 for 18/09/2023 is excluded from studies!
ISIN GB0006825383 for 18/09/2023 is excluded from studie

In [116]:
ftse_tidy_data =\
    pd\
        .DataFrame(target_rebal_prices)

ftse_tidy_data['post_date'] = pd.to_datetime(ftse_tidy_data['post_date'])


ftse_tidy_data =\
    ftse_tidy_data.merge(
        ftse_rebal[['ISIN', 'Post Date', 'FTSE100', 'FTSE250']],
        how = 'left',
        left_on = ['ISIN', 'post_date'],
        right_on = ['ISIN', 'Post Date']
    )

ftse_tidy_data.head()

/var/folders/mv/kbtg0d9s34s4w1xjw4w31b0h0000gn/T/ipykernel_80017/2702288160.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ftse_tidy_data['post_date'] = pd.to_datetime(ftse_tidy_data['post_date'])


,Name,ISIN,post_date,pre_twenty_pd,pre_five_pd,pre_three_pd,pre_one_pd,post_three_pd,post_five_pd,post_ten_pd,post_twenty_pd,Cov,Var,Beta,Corr,Post Date,FTSE100,FTSE250
0,ASOS,GB0030927254,2023-06-19,4.460999,3.292998,3.279998,3.703999,4.087000,3.927998,3.758999,3.518999,0.000101,0.000061,1.662526,0.412795,2023-06-19,NaN,-1.0
1,BRITISH LAND,GB0001367019,2023-06-19,3.562000,3.438999,3.347000,3.430000,3.083999,2.975000,3.132998,3.188999,0.000859,0.000102,8.439742,0.463406,2023-06-19,-1.0,1.0
2,CAPITA,GB00B23K0M20,2023-06-19,0.321000,0.329600,0.324400,0.320000,0.292000,0.279200,0.275000,0.278200,-0.000088,0.000021,-4.125385,-0.199032,2023-06-19,NaN,1.0
3,CAPRICORN ENERGY,GB00BRJ7R218,2023-06-19,1.950000,1.934000,1.920000,1.814000,1.818000,1.828000,1.878000,1.922000,-0.000423,0.000021,-19.837062,-0.195739,2023-06-19,NaN,-1.0
4,EMPIRIC STUDENT PROPERTY,GB00BLWDVR75,2023-06-19,0.916000,0.884000,0.869000,0.919000,0.860000,0.850000,0.840000,0.875000,0.000003,0.000287,0.009078,0.011979,2023-06-19,NaN,1.0


In [120]:

ftse_tidy_data.describe()

,post_date,pre_twenty_pd,pre_five_pd,pre_three_pd,pre_one_pd,post_three_pd,post_five_pd,post_ten_pd,post_twenty_pd,Cov,Var,Beta,Corr,Post Date,FTSE100,FTSE250
count,524,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,522.000000,524.000000,522.000000,522.000000,524,147.000000,512.000000
mean,2018-01-17 15:37:05.954198528,6.731242,6.606543,6.591596,6.556588,6.559066,6.540966,6.561822,6.591979,0.000314,0.000188,-1.208899,0.223618,2018-01-17 15:37:05.954198528,0.020408,0.007812
min,2013-03-18 00:00:00,0.132600,0.030000,0.022000,0.018000,0.017870,0.030000,0.038500,0.016000,-0.150046,0.000004,-1023.079460,-0.928859,2013-03-18 00:00:00,-1.000000,-1.000000
25%,2015-06-22 00:00:00,1.522250,1.468499,1.476875,1.466500,1.473000,1.469875,1.495000,1.538750,0.000002,0.000033,0.060637,0.019532,2015-06-22 00:00:00,-1.000000,-1.000000
50%,2018-03-19 00:00:00,3.075999,2.978998,2.984498,2.997499,3.029999,3.013998,3.000248,3.006498,0.000043,0.000065,0.763771,0.288238,2018-03-19 00:00:00,1.000000,1.000000
75%,2020-06-22 00:00:00,7.375500,7.172498,7.117500,7.159998,7.307498,7.092498,7.259999,7.288748,0.000154,0.000116,1.878989,0.544574,2020-06-22 00:00:00,1.000000,1.000000
max,2023-06-19 00:00:00,355.600080,347.600080,346.500000,343.100080,344.500000,343.000000,342.725080,344.500000,0.576266,0.005120,2293.681208,0.979215,2023-06-19 00:00:00,1.000000,1.000000
std,NaN,17.725827,17.300608,17.249200,17.068966,17.161728,17.146751,17.157087,17.125963,0.027530,0.000490,116.209725,0.424023,NaN,1.003210,1.000947
